In [2]:
!pip install tensorflow==2.16.1
!pip install keras==3.3.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found e

In [3]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)


TensorFlow version: 2.16.1
Keras version: 3.3.3


In [4]:
#Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from google.colab import files
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, GRU, Dropout, Dense, Flatten, Concatenate
import matplotlib.pyplot as plt

In [5]:
data = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Datasets/03-11/ALL.xlsx")

In [6]:
data['Label'] = data['Label'].map({'Attack': 0, 'Benign': 1})

In [7]:
X = data.drop(columns=['Label'])
sc = StandardScaler()
rescaled = sc.fit_transform(X)
y = data['Label']

In [8]:
from imblearn.over_sampling import ADASYN

ada = ADASYN(sampling_strategy='auto', random_state=42)
new_x, new_y = ada.fit_resample(rescaled, y)

In [9]:
from collections import Counter

counter_bef = Counter(y)
print("Before", counter_bef)

counter_aft = Counter(new_y)
print("Before", counter_aft)


Before Counter({0: 332855, 1: 93411})
Before Counter({0: 332855, 1: 332625})


In [10]:
X_train, X_test, y_train, y_test = train_test_split(new_x, new_y, test_size=0.33, random_state=42)

In [ ]:
# Define input shape
input_shape = (X_train.shape[1], 1)  # Assuming X_train is defined somewhere in your script

# Define the input layer
inputs = Input(shape=input_shape)

# Convolutional layers
x = inputs
for _ in range(4):
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)

# Flatten CNN output
cnn_out = Flatten()(x)

# GRU layers need to operate on sequences, so we need to prepare a sequence input
# Let's assume we reshape the same input for the GRU path, you might need to adjust this
y = Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape)(inputs)
y = MaxPooling1D(pool_size=2)(y)
for _ in range(2):
    y = GRU(units=64, activation='tanh', return_sequences=True)(y)
# Last GRU layer should not return sequences to match the flatten layer
gru_out = GRU(units=64, activation='tanh')(y)

# Concatenate CNN and GRU outputs
merged = Concatenate()([cnn_out, gru_out])

# Additional layers
x = Dense(units=64, activation='relu')(merged)
x = Dropout(0.5)(x)
outputs = Dense(units=2, activation='softmax')(x)  # Softmax for classification

# Create and compile model
model = Model(inputs=inputs, outputs=outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Reshape input data to fit the model
X_train_reshaped = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Train the model
history = model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_data=(X_test_reshaped, y_test))

model.save('IDS_Model.h5')
files.download('IDS_Model.h5')


plt.style.use('dark_background')

plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.suptitle('Optimizer : Adam', fontsize=10)
plt.ylabel('Loss', fontsize=16)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend(loc='upper right')

plt.subplot(1, 2, 2)
plt.ylabel('Accuracy', fontsize=16)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend(loc='lower right')
plt.show()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1740s 124ms/step - accuracy: 0.9887 - loss: 0.0409 - val_accuracy: 0.9962 - val_loss: 0.0170
Epoch 2/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1683s 121ms/step - accuracy: 0.9967 - loss: 0.0159 - val_accuracy: 0.9955 - val_loss: 0.0204
Epoch 3/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1617s 116ms/step - accuracy: 0.9970 - loss: 0.0148 - val_accuracy: 0.9973 - val_loss: 0.0133
Epoch 4/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1649s 117ms/step - accuracy: 0.9971 - loss: 0.0143 - val_accuracy: 0.9971 - val_loss: 0.0145
Epoch 5/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1528s 108ms/step - accuracy: 0.9969 - loss: 0.0149 - val_accuracy: 0.9973 - val_loss: 0.0142
Epoch 6/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1563s 112ms/step - accuracy: 0.9970 - loss: 0.0151 - val_accuracy: 0.9975 - val_loss: 0.0120
Epoch 7/10
13934/13934 ━━━━━━━━━━━━━━━━━━━━ 1625s 117ms/step - accuracy: 0.9971 - loss: 0.0132 - val_accuracy: 0.9974 - val_loss: 0.0134
Epoch 8/10
13934/13934 ━━━━━━━━━━━━━━━━━━

In [ ]:
import matplotlib.pyplot as plt

# Assuming history is obtained from the model.fit function
# Set the style of matplotlib to 'ggplot' for better visuals
plt.style.use('ggplot')

# Create a figure and a set of subplots
plt.figure(figsize=(14, 6))

# Plot training and validation loss
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st subplot
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Plot training and validation accuracy
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd subplot
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

# Show the plots
plt.show()
